In [1]:

16 px 7 depth 32-5 64-5 75ep --> 0.1552


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [ ]:
import numpy as np
np.random.seed(1984)

import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

from skimage import io
from skimage import transform
import skimage

from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.optimizers import SGD, Adagrad
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.constraints import maxnorm
from sklearn.metrics import log_loss
from keras import __version__ as keras_version
from collections import Counter

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [2]:
path1 = '/home/gs/DataScientist/planet'
trainPath = '/train-tif-64'
testPath = '/test-tif-64'

RESIZE = True
PIC_SIZE = 16
PIC_DEPTH = 7

def addChannels (i1):
    r = i1[:,:,0]
    g = i1[:,:,1]
    b = i1[:,:,2]
    nir = i1[:,:,3]
    evi = 2.5 * (nir - r) / (nir + 6 * r - 7.5 * b + 1)
    ndvi = (nir - r) / (nir.astype('float') + r)
    arvi = (nir - (2 * r - b)) / (nir.astype('float') + (2 * r + b) )
    res = np.dstack((i1, evi,ndvi,arvi))
    return(res)

def resizeImage (img):
    img = transform.resize(img, (PIC_SIZE, PIC_SIZE))
    return (img)


In [3]:
# read Y_train

try:
    Y_train = pd.read_csv(path1+'/train.csv')
except:
    path1 = '/home/ec2-user/DataScientist/planet'
    Y_train = pd.read_csv(path1+'/train.csv')

print (Y_train[0:5])

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in Y_train['tags'].values])))
label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}
print(label_map)
print
print(inv_label_map)

Y_trainDict = {}
for i, row in Y_train.iterrows():
    name = row['image_name']
    tags = row['tags']
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    Y_trainDict[name] = targets

print (Y_trainDict['train_0'])
print (Y_trainDict['train_1'])
print (Y_trainDict['train_2'])


  image_name                                       tags
0    train_0                               haze primary
1    train_1            agriculture clear primary water
2    train_2                              clear primary
3    train_3                              clear primary
4    train_4  agriculture clear habitation primary road
{'selective_logging': 16, 'cultivation': 8, 'clear': 1, 'habitation': 11, 'conventional_mine': 5, 'cloudy': 4, 'primary': 3, 'water': 6, 'haze': 7, 'slash_burn': 0, 'partly_cloudy': 9, 'artisinal_mine': 10, 'blooming': 2, 'bare_ground': 12, 'blow_down': 13, 'agriculture': 14, 'road': 15}

{0: 'slash_burn', 1: 'clear', 2: 'blooming', 3: 'primary', 4: 'cloudy', 5: 'conventional_mine', 6: 'water', 7: 'haze', 8: 'cultivation', 9: 'partly_cloudy', 10: 'artisinal_mine', 11: 'habitation', 12: 'bare_ground', 13: 'blow_down', 14: 'agriculture', 15: 'road', 16: 'selective_logging'}
[ 0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  1

In [4]:
def getImage(path):
    img = io.imread(path)
    return img

def load_train():
    X_train = []
    Y_train = []
    X_train_id = []
    start_time = time.time()

    print('Read train images')
    path = os.path.join(path1+trainPath, '*.tif')
    print (path)
    files = glob.glob(path)
    for fl in files:
        flbase = os.path.basename(fl)
        img = getImage(fl)
        if RESIZE == True:
            img = resizeImage(img)
        img = addChannels(img)
        X_train.append(img)
        name = flbase.replace('.tif', '')
        X_train_id.append(name)
        Y_train.append(Y_trainDict[name])

    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_train,X_train_id, Y_train

def load_test():
    X_test = []
    X_test_id = []
    start_time = time.time()

    print('Read test images')
    path = os.path.join(path1+testPath, '*.tif')
    print (path)
    files = glob.glob(path)
    for fl in files:
        flbase = os.path.basename(fl)
        img = getImage(fl)
        if RESIZE == True:
            img = resizeImage(img)
        img = addChannels(img)
        X_test.append(img)
        name = flbase.replace('.tif', '')
        X_test_id.append(name)

    print('Read test data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_test, X_test_id

def mapf (arr):
    res = ''
    for i in range(0,17):
        if arr[i] > 0.2:
            res += inv_label_map[i] + ' '
    res = res.rstrip()
    return res
 

In [5]:
X_train, X_train_id, Y_train = load_train()
print (len(X_train), len(X_train_id))
print(X_train_id[0:5])
print(X_train[0].shape)
print(Y_train[0])
print(Y_train[1])

X_test, X_test_id = load_test()
print (len(X_test), len(X_test_id))
print(X_test_id[0:5])
print(X_test[0].shape)


Read train images
/home/gs/DataScientist/planet/train-tif-64/*.tif
Read train data time: 125.83 seconds
(40479, 40479)
['train_8028', 'train_5846', 'train_26698', 'train_7878', 'train_25904']
(16, 16, 7)
[ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.]
[ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Read test images
/home/gs/DataScientist/planet/test-tif-64/*.tif
Read test data time: 116.41 seconds
(40669, 40669)
['test_34045', 'test_32937', 'test_10537', 'test_30945', 'test_13470']
(16, 16, 7)


In [6]:
#
# SAVE #
#

# save data frames for later use

# Y_trainAll
# X_train
# X_test

import pickle
#pickle.dump( Y_trainAll, open( "SAVE_yTrainAll.p", "wb" ) )
pickle.dump( X_train, open( "SAVE_xTrain.p", "wb" ) )
pickle.dump( Y_train, open( "SAVE_yTrain.p", "wb" ) )
pickle.dump( X_test, open( "SAVE_xTest.p", "wb" ) )
pickle.dump( X_test_id, open( "SAVE_xTestId.p", "wb" ) )
pickle.dump( Y_trainDict, open( "SAVE_yTrainDict.p", "wb" ) )
pickle.dump( label_map, open( "SAVE_labelMap.p", "wb" ) )
pickle.dump( inv_label_map, open( "SAVE_invLabelMap.p", "wb" ) )



In [14]:
#
# LOAD
#
# load data
import pickle
#Y_trainAll = pickle.load( open( "SAVE_yTrainAll.p", "rb" ) )
X_train = pickle.load( open( "SAVE_xTrain.p", "rb" ) )
Y_train = pickle.load( open( "SAVE_yTrain.p", "rb" ) )
X_test = pickle.load( open( "SAVE_xTest.p", "rb" ) )
X_test_id = pickle.load( open( "SAVE_xTestId.p", "rb" ) )
Y_trainDict = pickle.load( open( "SAVE_yTrainDict.p", "rb" ) )
label_map = pickle.load( open( "SAVE_labelMap.p", "rb" ) )
inv_label_map = pickle.load( open( "SAVE_invLabelMap.p", "rb" ) )




In [15]:

print ('convert...')
Y_train = np.array(Y_train, np.uint8)
X_train = np.array(X_train, np.float16)
X_test = np.array(X_test, np.float16) 

print (type(X_train))
print (type(Y_train))
print (type(X_test))


convert...
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


In [16]:
print (np.isnan(X_train).sum())
print (np.isnan(X_test).sum())
print (np.isinf(X_train).sum())
print (np.isinf(X_test).sum())
print (np.isneginf(X_train).sum())
print (np.isneginf(X_test).sum())



1536
512
12
11
9
6


In [17]:
# find NAN

X_train [np.isnan(X_train)] = 0
X_train [np.isinf(X_train)] = 0
X_train [np.isneginf(X_train)] = 0

X_test [np.isnan(X_test)] = 0
X_test [np.isinf(X_test)] = 0
X_test [np.isneginf(X_test)] = 0
print ('done.')

done.


In [18]:

print ('normalize...')
for i in range(0,7):
    print (i)
    mi = min( np.min(X_train[:,:,:,i]), np.min(X_test[:,:,:,i]) )
    ma = max( np.max(X_train[:,:,:,i]), np.max(X_test[:,:,:,i]) )
    print (ma)
    print (mi)
    print (ma - mi * 1.0)
    print ('')
    X_train[:,:,:,i] = (X_train[:,:,:,i] - mi * 1.0) / (ma - mi * 1.0)
    X_test[:,:,:,i]  = (X_test[:,:,:,i] - mi * 1.0) / (ma - mi * 1.0)

X_train [np.isnan(X_train)] = 0
X_train [np.isinf(X_train)] = 0
X_train [np.isneginf(X_train)] = 0

X_test [np.isnan(X_test)] = 0
X_test [np.isinf(X_test)] = 0
X_test [np.isneginf(X_test)] = 0
    
    
print ('train')
print (np.min(X_train))
print (np.mean(X_train))
print (np.max(X_train))
    
print ('test')
print (np.min(X_test))
print (np.mean(X_test))
print (np.max(X_test))
    
    
print ('shapes')
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

normalize...
0
33952.0
0.0
33952.0

1
25088.0
0.0
25088.0

2
40000.0
0.0
40000.0

3
38784.0
0.0
38784.0

4
63552.0
-41504.0
105056.0

5
1.0
-0.90918
1.9091796875

6
1.0
-0.57959
1.57958984375

train
0.0
0.26343
1.0
test
0.0
0.26367
1.0
shapes
(40479, 16, 16, 7)
(40479, 17)
(40669, 16, 16, 7)


In [20]:
# train / use larger split value for predictions, more epochs

split = 35000
x_train, x_valid, y_train, y_valid = X_train[:split], X_train[split:], Y_train[:split], Y_train[split:]

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(PIC_SIZE, PIC_SIZE, PIC_DEPTH)))

model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#model.add(Conv2D(128, (5, 5), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer='adam',
              metrics=['accuracy'])
              
model.fit(x_train, y_train,
          batch_size=128,
          epochs=75,
          verbose=1,
          validation_data=(x_valid, y_valid))
          
from sklearn.metrics import fbeta_score

p_valid = model.predict(x_valid, batch_size=128)
#print(y_valid)
#print(p_valid)
print(fbeta_score(y_valid, np.array(p_valid) > 0.5, beta=2, average='macro'))

Train on 35000 samples, validate on 5479 samples
Epoch 1/75
35000/35000 [==============================] - 286s - loss: 0.2698 - acc: 0.8979 - val_loss: 0.2266 - val_acc: 0.9107

In [ ]:
# predict

preds = model.predict(X_test, batch_size=128)


In [ ]:
# X_test_id, preds -> submission
predsText = []
for e in preds:
    predsText.append(mapf(e))
print (predsText[0:5])

In [ ]:
#image_name,tags
#test_0,primary clear agriculture road water

subName = path1 + '/SUB_04_64pix_xxep.csv'
f = open(subName, 'w')
f.write('image_name,tags\n')
for i in range(0,len(predsText)):
    f.write(X_test_id[i]+','+predsText[i]+'\n')
f.close()


In [ ]:
print (type(preds))
raw = pd.DataFrame(preds)
raw['id'] = X_test_id
print (raw.head())
raw.to_csv('RAW_04_64px_xxep.csv', index=False)